In [1]:
import pandas as pd
import numpy as np
# keras 2.0.2
from keras.layers import Input, Dense, Dropout
from keras.layers.advanced_activations import LeakyReLU, PReLU
from keras.models import Model, Sequential
from keras.utils import np_utils
from keras.optimizers import Adam, RMSprop, SGD
import keras.backend as K
import os, sys
sys.path.append('utils/')
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import KFold
from utils import *
from evaluation import compute_ks, compute_cvm, roc_auc_truncated
from tqdm import tqdm, trange
from keras.models import model_from_json

SEED = 5

pd.set_option('display.max_columns', 100)

%matplotlib inline

Using TensorFlow backend.
/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


## Utils

In [2]:
def dump_model(model, model_path, weigths_path):
    model_json = model.to_json()
    with open(model_path, "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    model.save_weights(weigths_path)

def load_model(model_path, weigths_path, optimizer=RMSprop()):
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    # load weights into new model
    model.load_weights(weigths_path)
    model.compile(optimizer=optimizer,
                      loss='categorical_crossentropy',
                      metrics=['acc'])
    return model

# Preparing data

In [3]:
label_prediction = pd.read_csv('../datasets/training.csv.zip').drop('id', 1)
features = label_prediction.drop(['signal', 'mass', 'min_ANNmuon', 'production'], 1).columns
print(label_prediction.shape)
label_prediction.head()

(67553, 50)


,LifeTime,dira,FlightDistance,FlightDistanceError,IP,IPSig,VertexChi2,pt,DOCAone,DOCAtwo,DOCAthree,IP_p0p2,IP_p1p2,isolationa,isolationb,isolationc,isolationd,isolatione,isolationf,iso,CDF1,CDF2,CDF3,ISO_SumBDT,p0_IsoBDT,p1_IsoBDT,p2_IsoBDT,p0_track_Chi2Dof,p1_track_Chi2Dof,p2_track_Chi2Dof,p0_IP,p1_IP,p2_IP,p0_IPSig,p1_IPSig,p2_IPSig,p0_pt,p1_pt,p2_pt,p0_p,p1_p,p2_p,p0_eta,p1_eta,p2_eta,SPDhits,production,signal,mass,min_ANNmuon
0,0.001578,0.999999,14.033335,0.681401,0.016039,0.451886,1.900433,1482.037476,0.066667,0.060602,0.083660,0.208855,0.074343,8,5,7,1.0,0.0,3.0,4.0,0.473952,0.349447,0.329157,-0.579324,-0.256309,-0.215444,-0.107570,1.921700,0.866657,1.230708,0.988054,0.601483,0.277090,16.243183,4.580875,5.939936,353.819733,448.369446,1393.246826,3842.096436,12290.760742,39264.398438,3.076006,4.003800,4.031514,458,-99,0,1866.300049,0.277559
1,0.000988,0.999705,5.536157,0.302341,0.142163,9.564503,0.865666,3050.720703,0.024022,0.019245,0.030784,0.336345,0.173161,7,12,2,0.0,1.0,1.0,2.0,0.325785,0.265939,0.192599,-0.873926,-0.223774,-0.224871,-0.425281,0.958776,0.858357,1.810709,0.098752,0.219099,0.614524,3.610463,15.555593,11.238523,656.524902,2033.918701,747.137024,8299.368164,16562.667969,7341.257812,3.228553,2.786543,2.975564,406,-99,0,1727.095947,0.225924
2,0.000877,0.999984,6.117302,0.276463,0.034746,1.970751,10.975849,3895.908691,0.055044,0.047947,0.096829,0.169165,0.079789,1,0,1,0.0,0.0,0.0,0.0,1.000000,0.786482,0.557760,-0.479636,-0.202451,-0.100762,-0.176424,0.720973,1.408519,1.038347,0.186143,0.215668,0.371820,4.851371,11.590331,13.723293,658.523743,2576.380615,963.652466,11323.134766,22695.388672,10225.309570,3.536903,2.865686,3.052810,196,-99,0,1898.588013,0.368630
3,0.000854,0.999903,5.228067,0.220739,0.076389,4.271331,3.276358,4010.781738,0.053779,0.006417,0.044816,0.050989,0.068167,2,2,4,0.0,0.0,0.0,0.0,1.000000,0.501195,0.501195,-0.439453,-0.162267,-0.176424,-0.100762,1.172767,2.044164,0.811454,0.255752,0.210698,0.392195,7.292110,8.778173,16.462036,1047.216187,1351.734131,1685.003662,11502.081055,16909.515625,9141.426758,3.087461,3.218034,2.375592,137,-99,0,1840.410034,0.246045
4,0.001129,0.999995,39.069534,1.898197,0.120936,4.984982,0.468348,4144.546875,0.004491,0.037326,0.019026,0.172065,0.131732,0,2,0,0.0,0.0,0.0,0.0,1.000000,1.000000,0.639926,-0.822285,-0.291524,-0.261078,-0.269682,1.523252,0.435325,0.581312,0.270755,0.183355,0.630763,6.783962,3.342091,17.252840,1442.538208,1755.792236,1282.428711,74117.117188,97612.804688,47118.785156,4.632295,4.711155,4.296878,477,-99,0,1899.793945,0.222060


In [4]:
test = pd.read_csv('../datasets/test.csv.zip')
print(test.shape)

(855819, 47)


In [5]:
private_dataset = pd.merge(test, pd.read_csv('../datasets/private_eval.csv'), on='id').drop_duplicates()
print(private_dataset.shape)
private_dataset.head()

(28497, 48)


,id,LifeTime,dira,FlightDistance,FlightDistanceError,IP,IPSig,VertexChi2,pt,DOCAone,DOCAtwo,DOCAthree,IP_p0p2,IP_p1p2,isolationa,isolationb,isolationc,isolationd,isolatione,isolationf,iso,CDF1,CDF2,CDF3,ISO_SumBDT,p0_IsoBDT,p1_IsoBDT,p2_IsoBDT,p0_track_Chi2Dof,p1_track_Chi2Dof,p2_track_Chi2Dof,p0_IP,p1_IP,p2_IP,p0_IPSig,p1_IPSig,p2_IPSig,p0_pt,p1_pt,p2_pt,p0_p,p1_p,p2_p,p0_eta,p1_eta,p2_eta,SPDhits,signal
0,4369817,0.002109,0.999998,14.665833,0.134259,0.029963,2.091192,0.446515,6850.274414,0.008145,0.008414,0.005803,0.478501,0.182662,1,5,2,0.0,0.0,0.0,0.0,0.963404,0.941582,0.832183,-1.271832,-0.431318,-0.429426,-0.411087,1.040657,1.374021,0.888281,0.541408,0.812435,0.449803,24.295046,40.261230,23.897257,1691.214111,3163.742188,2060.649170,11679.781250,14723.319336,14744.435547,2.620276,2.219076,2.656073,353,1
1,10213012,0.000886,0.999992,9.180307,0.472137,0.037329,2.025650,7.784008,4024.888916,0.070422,0.020018,0.078934,0.130110,0.185814,1,2,1,0.0,0.0,0.0,0.0,0.621012,0.621012,0.621012,-0.929209,-0.171253,-0.391201,-0.366756,1.097838,0.505729,0.969616,0.278849,0.215638,0.306633,9.881294,9.835187,9.932581,1413.249878,1814.381470,1084.320923,22417.171875,20466.382812,18283.640625,3.456087,3.114216,3.517320,52,1
2,17806160,0.001313,0.999966,17.477598,1.382421,0.157643,6.020585,1.043844,1584.708984,0.010628,0.009894,0.068503,0.375108,0.232583,5,3,3,1.0,1.0,1.0,3.0,0.499311,0.396624,0.396624,-0.385559,-0.084681,-0.124454,-0.176424,1.382239,1.274481,0.885738,0.280228,0.179730,1.388035,5.037262,5.983574,10.372314,493.309265,1422.916504,317.407227,12352.144531,59287.277344,6400.515137,3.913197,4.422689,3.696480,202,1
3,641074,0.000827,0.999993,5.873784,0.162675,0.022190,2.182303,4.583155,7745.390137,0.011685,0.012809,0.022599,0.157218,0.066670,2,0,6,0.0,0.0,0.0,0.0,0.915044,0.854287,0.725930,-1.448177,-0.524151,-0.539526,-0.384501,0.870120,1.070111,1.276760,0.259480,0.335885,0.128558,10.903188,24.863413,10.396156,1322.295776,2413.235840,4147.405762,6593.108887,15263.637695,20328.482422,2.289592,2.531343,2.272115,211,1
4,11383293,0.001149,0.999999,9.548421,0.384930,0.014353,0.766637,0.499251,2880.133545,0.017162,0.001223,0.039240,0.185238,0.066593,2,6,1,0.0,1.0,0.0,1.0,0.921500,0.844762,0.443685,-0.869419,-0.432826,-0.188276,-0.248317,1.071941,1.061785,0.953241,1.055305,0.234065,0.158303,18.587053,11.259723,4.993015,392.172211,1645.024414,1098.795898,3348.355713,19929.757812,25797.041016,2.834224,3.185898,3.848738,251,1


In [6]:
public_dataset = pd.merge(test, pd.read_csv('../datasets/public_eval.csv'), on='id').drop_duplicates()
print(public_dataset.shape)
public_dataset.head()

(12302, 48)


,id,LifeTime,dira,FlightDistance,FlightDistanceError,IP,IPSig,VertexChi2,pt,DOCAone,DOCAtwo,DOCAthree,IP_p0p2,IP_p1p2,isolationa,isolationb,isolationc,isolationd,isolatione,isolationf,iso,CDF1,CDF2,CDF3,ISO_SumBDT,p0_IsoBDT,p1_IsoBDT,p2_IsoBDT,p0_track_Chi2Dof,p1_track_Chi2Dof,p2_track_Chi2Dof,p0_IP,p1_IP,p2_IP,p0_IPSig,p1_IPSig,p2_IPSig,p0_pt,p1_pt,p2_pt,p0_p,p1_p,p2_p,p0_eta,p1_eta,p2_eta,SPDhits,signal
0,12200521,0.001825,0.999989,8.118148,0.198013,0.038373,2.368293,3.995679,3088.533447,0.016940,0.050317,0.014406,0.177796,0.290612,4,5,1,1.0,0.0,0.0,1.0,0.601583,0.601583,0.252849,-1.112244,-0.232027,-0.425281,-0.454935,0.991786,0.519075,1.587669,0.504891,0.323959,0.729432,21.858988,19.899220,23.709482,1346.735229,1385.601074,612.744080,8325.249023,8831.709961,9389.864258,2.508150,2.539151,3.421520,433,1
1,2491177,0.000535,0.999934,8.240436,1.095218,0.083408,4.210687,8.162792,3235.502197,0.095860,0.070505,0.067769,0.054672,0.140425,1,5,6,0.0,1.0,1.0,2.0,0.386806,0.278460,0.257121,-0.775315,-0.284896,-0.229474,-0.260944,1.142997,0.980602,1.558769,0.186751,0.147826,0.303290,5.095261,6.199450,3.797090,1509.661865,1778.513550,352.617950,45959.210938,41672.800781,6638.411621,4.108746,3.846762,3.627684,369,0
2,14699668,0.000850,0.999996,28.220333,0.677022,0.077940,8.915274,0.035283,13247.209961,0.000482,0.002346,0.002073,0.174334,0.094599,2,2,2,2.0,3.0,3.0,8.0,0.633866,0.323418,0.316222,-0.364786,-0.124454,-0.115879,-0.124454,0.771244,0.681871,0.886693,0.079222,0.292620,0.273358,3.897521,18.936714,30.215090,2613.442871,4030.838379,6680.432617,38441.621094,60690.246094,88079.054688,3.380462,3.403851,3.270758,414,0
3,225705,0.001145,1.000000,16.196651,0.395021,0.009561,0.717381,2.758028,5904.848633,0.000262,0.030797,0.008239,0.188568,0.084972,1,4,1,0.0,0.0,0.0,0.0,0.706804,0.619967,0.579491,-1.177279,-0.482754,-0.396888,-0.297637,1.010778,1.088703,0.871755,0.443124,0.364227,0.270769,16.680660,22.491060,15.464107,1185.568970,2531.938477,2261.802734,14468.316406,27498.492188,41622.230469,3.193202,3.076167,3.604880,149,1
4,4323792,0.001371,0.999999,18.812433,0.529717,0.029653,1.518456,4.937074,4474.023438,0.048851,0.030970,0.095518,0.278755,0.083573,0,1,0,0.0,0.0,0.0,0.0,0.865771,0.825148,0.750697,-1.041557,-0.421370,-0.198817,-0.421370,1.265123,0.932610,1.042837,0.453670,0.494659,0.305528,13.372378,24.646584,7.905024,555.046448,2260.256836,1716.283325,8806.927734,28214.091797,44662.679688,3.456395,3.215882,3.951755,163,1


In [7]:
check_agreement = pd.read_csv('../datasets/check_agreement.csv.zip')

print(check_agreement.shape)
check_agreement.head()

(331147, 49)


,id,LifeTime,dira,FlightDistance,FlightDistanceError,IP,IPSig,VertexChi2,pt,DOCAone,DOCAtwo,DOCAthree,IP_p0p2,IP_p1p2,isolationa,isolationb,isolationc,isolationd,isolatione,isolationf,iso,CDF1,CDF2,CDF3,ISO_SumBDT,p0_IsoBDT,p1_IsoBDT,p2_IsoBDT,p0_track_Chi2Dof,p1_track_Chi2Dof,p2_track_Chi2Dof,p0_IP,p1_IP,p2_IP,p0_IPSig,p1_IPSig,p2_IPSig,p0_pt,p1_pt,p2_pt,p0_p,p1_p,p2_p,p0_eta,p1_eta,p2_eta,SPDhits,signal,weight
0,15347063,0.001451,0.999964,6.945030,0.229196,0.058117,2.961298,7.953543,2251.611816,0.082219,0.084005,0.066887,0.185107,0.214719,8,6,1,2.0,1.0,1.0,4.0,0.732076,0.492269,0.179091,-0.207475,-0.019306,-0.089797,-0.098372,0.606178,0.862549,1.487057,0.483199,0.474925,0.426797,24.701061,10.732132,8.853514,1438.064697,468.645721,834.562378,10392.814453,6380.673828,15195.594727,2.666142,3.302978,3.594246,512,0,-0.307813
1,14383299,0.000679,0.999818,9.468235,0.517488,0.189683,14.413060,7.141451,10594.470703,0.007983,0.044154,0.001321,0.039357,0.217507,5,6,17,1.0,1.0,1.0,3.0,0.802508,0.605835,0.584701,-0.659644,-0.278330,-0.186370,-0.194944,1.900118,1.073474,1.336784,0.712242,0.260311,0.123877,11.312134,16.435398,7.737038,316.791351,7547.703613,2861.309814,3174.356934,64480.023438,23134.953125,2.995265,2.834816,2.779366,552,0,-0.331421
2,7382797,0.003027,0.999847,13.280714,0.219291,0.231709,11.973175,4.778880,2502.196289,0.045085,0.106614,0.005850,0.335788,0.885080,2,2,1,0.0,0.0,1.0,1.0,0.682607,0.682607,0.295038,-0.399239,-0.115879,-0.131069,-0.152291,0.660675,1.683084,0.798658,0.381544,1.163556,1.290409,16.435801,20.686119,44.521961,1887.477905,317.579529,932.128235,15219.761719,3921.181641,10180.791016,2.776633,3.204923,3.081832,318,0,-0.382215
3,6751065,0.000810,0.999998,5.166821,0.167886,0.011298,0.891142,5.528002,5097.813965,0.055115,0.038642,0.003864,0.076522,0.068347,4,4,3,0.0,0.0,0.0,0.0,0.533615,0.533615,0.533615,-0.821041,-0.208248,-0.177802,-0.434991,0.770563,1.093031,0.938619,0.564650,0.164411,0.166646,24.878387,7.873435,9.630725,975.041687,1650.837524,2617.248291,4365.082520,13221.149414,24291.875000,2.179345,2.769762,2.918251,290,0,1.465194
4,9439580,0.000706,0.999896,10.897236,0.284975,0.160511,16.367550,8.670339,20388.097656,0.015587,0.020872,0.014612,0.249906,0.139937,0,1,0,0.0,0.0,0.0,0.0,0.926410,0.926410,0.926410,-1.116815,-0.328938,-0.443564,-0.344313,1.080559,1.471946,1.123868,0.373736,0.230584,0.112430,28.557213,18.738485,7.389726,6035.000977,9657.492188,4763.682617,27463.011719,46903.394531,24241.628906,2.196114,2.262732,2.310401,45,0,-0.477084


In [8]:
check_correlation = pd.read_csv('../datasets/check_correlation.csv.zip')

print(check_correlation.shape)
check_correlation.head()

(5514, 48)


,id,LifeTime,dira,FlightDistance,FlightDistanceError,IP,IPSig,VertexChi2,pt,DOCAone,DOCAtwo,DOCAthree,IP_p0p2,IP_p1p2,isolationa,isolationb,isolationc,isolationd,isolatione,isolationf,iso,CDF1,CDF2,CDF3,ISO_SumBDT,p0_IsoBDT,p1_IsoBDT,p2_IsoBDT,p0_track_Chi2Dof,p1_track_Chi2Dof,p2_track_Chi2Dof,p0_IP,p1_IP,p2_IP,p0_IPSig,p1_IPSig,p2_IPSig,p0_pt,p1_pt,p2_pt,p0_p,p1_p,p2_p,p0_eta,p1_eta,p2_eta,SPDhits,mass
0,11120335,0.000703,0.999715,2.927074,0.214014,0.081302,4.259793,1.066585,3108.189941,0.010767,0.024147,0.003066,0.100619,0.087596,2,10,5,2.0,1.0,1.0,4.0,0.661286,0.592280,0.592280,-0.229346,-0.089797,-0.049752,-0.089797,1.015762,0.918588,2.555963,0.336053,0.138520,0.239775,6.975648,7.199831,9.638602,1142.124512,1294.450928,1073.976440,9274.671875,7963.914062,6712.897949,2.783731,2.503310,2.519349,280,1723.887939
1,11495369,0.000601,0.999950,15.849142,0.842973,0.182213,13.882857,5.780046,6858.264648,0.007574,0.020010,0.064194,0.058354,0.247012,3,3,4,0.0,1.0,1.0,2.0,0.744360,0.441476,0.209664,-0.364786,-0.115879,-0.124454,-0.124454,1.413934,0.696266,1.531379,0.282444,0.298322,0.168395,10.620588,17.345636,7.344349,1174.860229,3981.284912,1859.680542,22844.791016,109955.101562,37051.800781,3.660059,4.011287,3.684429,386,1926.284058
2,7098902,0.002009,0.999984,43.358494,1.323199,0.200158,12.870687,3.460782,3604.347412,0.052849,0.127547,0.073476,0.365799,0.292673,0,1,0,0.0,0.0,0.0,0.0,0.699672,0.678214,0.437321,-1.175178,-0.351104,-0.414748,-0.409326,1.307686,1.081815,0.674668,0.173122,0.480514,1.041596,4.409735,30.871080,7.409998,1046.867310,2564.430420,354.095032,45214.070312,77265.429688,12087.007812,4.458619,4.098382,4.223254,433,1830.873047
3,8103692,0.001268,0.999550,6.910733,0.198652,0.167730,8.559438,7.676139,3240.960449,0.101368,0.002148,0.037449,0.060147,0.162985,4,9,4,1.0,1.0,1.0,3.0,0.257655,0.184313,0.182857,-0.495822,-0.194944,-0.176424,-0.124454,1.019122,2.248540,1.243463,0.390963,0.627191,0.136959,11.687304,31.343948,4.312659,1051.328979,1178.332031,1285.701660,11245.551758,6770.969238,17003.119141,3.060873,2.434033,3.273807,245,1909.119019
4,10160864,0.001937,0.999996,36.135208,0.508036,0.097170,9.295684,0.426340,6448.445312,0.007005,0.008510,0.009841,0.481759,0.455559,0,4,0,0.0,0.0,0.0,0.0,0.588020,0.537217,0.222616,-0.766796,-0.263475,-0.263475,-0.239847,0.766173,0.715839,0.847636,0.612008,0.498065,0.641609,38.172073,38.144756,14.099512,3081.089111,3140.512207,385.343475,48171.457031,43973.835938,7368.522949,3.441606,3.331079,3.643300,489,1600.925049


In [57]:
Xt_train, Xt_test, yt_train, yt_test = train_test_split(label_prediction[features].values, label_prediction.signal.values, train_size=0.8)
domain_adaptation_random = pd.read_csv('../datasets/domain_adaptation_random.csv').sample(Xt_train.shape[0], random_state=SEED)
domain_adaptation_high_weight = pd.read_csv('../datasets/domain_adaptation_high_weight.csv').sample(Xt_train.shape[0], random_state=SEED)
Xd_1, yd_1 = domain_adaptation_high_weight[features].values, domain_adaptation_high_weight.domain.values
Xd_2, yd_2 = domain_adaptation_random[features].values, domain_adaptation_random.domain.values
Xm, ym = check_correlation.sample(Xt_train.shape[0], replace=True)[features].values, check_correlation.sample(Xt_train.shape[0], replace=True).mass.values

X_public, y_public = public_dataset[features].values, public_dataset.signal.values
X_private, y_private = private_dataset[features].values, private_dataset.signal.values

print(Xd_1.shape, yd_1.shape)
print(Xd_2.shape, yd_2.shape)
print(Xm.shape, ym.shape)
print(Xt_train.shape, yt_train.shape)
print(Xt_test.shape, yt_test.shape)
print(X_public.shape, y_public.shape)
print(X_private.shape, y_private.shape)

((54042, 46), (54042,))
((54042, 46), (54042,))
((54042, 46), (54042,))
((54042, 46), (54042,))
((13511, 46), (13511,))
((12302, 46), (12302,))
((28497, 46), (28497,))


## Cross-validation function

In [10]:
def cross_validation(model):
    agreement_probs = model.predict(check_agreement[features].values)[:, 1]
    correlation_probs = model.predict(check_correlation[features].values)[:, 1]
    auc_holdout = roc_auc_truncated(yt_test, model.predict(Xt_test)[:, 1])
    auc_public = roc_auc_truncated(y_public, model.predict(X_public)[:, 1])
    auc_private = roc_auc_truncated(y_private, model.predict(X_private)[:, 1])
    ks = compute_ks(agreement_probs[check_agreement['signal'].values == 0],
        agreement_probs[check_agreement['signal'].values == 1],
        check_agreement[check_agreement['signal'] == 0]['weight'].values,
        check_agreement[check_agreement['signal'] == 1]['weight'].values)
    cvm = compute_cvm(correlation_probs, check_correlation['mass'].values)
    return auc_holdout, auc_public, auc_private, ks, cvm

## Building model

In [11]:
def build_model(optimizer=RMSprop()):
    # this returns a tensor
    inputs = Input(shape=(len(features),))

    # a layer instance is callable on a tensor, and returns a tensor
    x = Dense(70)(inputs)
    x = PReLU()(x)
    
    x = Dense(35)(x)
    x = PReLU()(x)

    x = Dense(20)(x)
    x = PReLU()(x)

    x = Dropout(0.09)(x)
    x = Dense(10)(x)
    x = PReLU()(x)

    x = Dropout(0.07)(x)
    x = Dense(7)(x)
    x = PReLU()(x)

    p = Dense(2, activation="softmax")(x)

    # this creates a model that includes
    # the Input layer and three Dense layers
    model = Model(input=inputs, output=p)

    model.compile(optimizer=optimizer,
                      loss='categorical_crossentropy',
                      metrics=['acc'])
    return model

In [25]:
model = build_model(optimizer='sgd')
model.fit(Xt_train, np_utils.to_categorical(yt_train), nb_epoch=1, batch_size=320)
preds = model.predict(Xt_test)[:, 1]
print('AUC: %.4f' % roc_auc_truncated(yt_test, preds))

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:27: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:2: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  from ipykernel import kernelapp as app


Epoch 1/1
54042/54042 [==============================] - 1s - loss: 9.9179 - acc: 0.3847     
AUC: 0.5000


You can see that SGD converges to wrong minima. That's because wrong variables initialized. Lets try Adam optimization and more initializations

In [26]:
print('Initializing')
best_auc = 0.5
iter_ = 0
max_iter = 30
best_model = None
while best_auc < 0.7 or iter_ < max_iter:
    model = build_model(optimizer='rmsprop')
    model.fit(Xt_train, np_utils.to_categorical(yt_train), nb_epoch=1, batch_size=320)
    preds = model.predict(Xt_test)[:, 1]
    auc = roc_auc_truncated(yt_test, preds)
    iter_ += 1
    print('AUC: %.4f' % auc)
    if auc > best_auc:
        best_auc = auc
        best_model = model

Initializing


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:27: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("de..., inputs=Tensor("in...)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:8: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.


Epoch 1/1
54042/54042 [==============================] - 1s - loss: 6.2891 - acc: 0.6096     
AUC: 0.5000
Epoch 1/1
54042/54042 [==============================] - 1s - loss: 6.7650 - acc: 0.5786     
AUC: 0.5000
Epoch 1/1
54042/54042 [==============================] - 1s - loss: 6.1853 - acc: 0.6058     
AUC: 0.5010
Epoch 1/1
54042/54042 [==============================] - 1s - loss: 6.1967 - acc: 0.6155     
AUC: 0.5000
Epoch 1/1
54042/54042 [==============================] - 1s - loss: 2.3061 - acc: 0.5695     
AUC: 0.7036
Epoch 1/1
54042/54042 [==============================] - 1s - loss: 9.8601 - acc: 0.3881     
AUC: 0.5000
Epoch 1/1
54042/54042 [==============================] - 1s - loss: 6.2828 - acc: 0.6099     
AUC: 0.5000
Epoch 1/1
54042/54042 [==============================] - 1s - loss: 1.7337 - acc: 0.5467     
AUC: 0.7641
Epoch 1/1
54042/54042 [==============================] - 1s - loss: 7.9641 - acc: 0.5017     
AUC: 0.5000
Epoch 1/1
54042/54042 [=======================

Let's serialize model weights

In [27]:
dump_model(best_model, model_path='model.json', weigths_path='model.h5')

Let's try to init variables with batches with mass

In [36]:
best_model = load_model(model_path='model.json', weigths_path="model.h5")

best_model.fit(Xt_train, np_utils.to_categorical(yt_train), nb_epoch=15, batch_size=64)
preds = best_model.predict(Xt_test)[:, 1]
roc_auc_truncated(yt_test, preds)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  app.launch_new_instance()


Epoch 1/15
54042/54042 [==============================] - 6s - loss: 0.7058 - acc: 0.6054     
Epoch 2/15
54042/54042 [==============================] - 5s - loss: 0.6680 - acc: 0.6157     
Epoch 3/15
54042/54042 [==============================] - 5s - loss: 0.6662 - acc: 0.6186     
Epoch 4/15
54042/54042 [==============================] - 5s - loss: 0.6540 - acc: 0.6270     
Epoch 5/15
54042/54042 [==============================] - 5s - loss: 0.6288 - acc: 0.6679     
Epoch 6/15
54042/54042 [==============================] - 6s - loss: 0.6083 - acc: 0.6851     
Epoch 7/15
54042/54042 [==============================] - 5s - loss: 0.5929 - acc: 0.6973     
Epoch 8/15
54042/54042 [==============================] - 5s - loss: 0.5791 - acc: 0.7064     
Epoch 9/15
54042/54042 [==============================] - 5s - loss: 0.5699 - acc: 0.7151     
Epoch 10/15
54042/54042 [==============================] - 5s - loss: 0.5601 - acc: 0.7202     
Epoch 11/15
54042/54042 [========================

0.96316734971244466

In [37]:
auc_holdout, auc_public, auc_private, ks, cvm = cross_validation(best_model)
print ('AUC (holdout): %.2f' % auc_holdout)
print ('AUC (public leaderboard): %.2f' % auc_public)
print ('AUC (private leaderboard): %.2f' % auc_private)
print ('KS: %.2f' % ks)
print ('CvM: %.4f' % cvm)

AUC (holdout): 0.96
AUC (public leaderboard): 0.96
AUC (private leaderboard): 0.96
KS: 0.17
CvM: 0.0008


In [38]:
dump_model(best_model, 'model_trained.json', 'model_trained.h5')

## Domain adaptation

In [39]:
def cross_validation_dann(model):
    agreement_probs = model.predict([check_agreement[features].values, np.zeros_like(check_agreement[features].values)])[0][:, 1]
    correlation_probs = model.predict([check_correlation[features].values, np.zeros_like(check_correlation[features].values)])[0][:, 1]
    auc_holdout = roc_auc_truncated(yt_test, model.predict([Xt_test, np.zeros_like(Xt_test)])[0][:, 1])
    auc_public = roc_auc_truncated(y_public, model.predict([X_public, np.zeros_like(X_public)])[0][:, 1])
    auc_private = roc_auc_truncated(y_private, model.predict([X_private, np.zeros_like(X_private)])[0][:, 1])
    ks = compute_ks(agreement_probs[check_agreement['signal'].values == 0],
        agreement_probs[check_agreement['signal'].values == 1],
        check_agreement[check_agreement['signal'] == 0]['weight'].values,
        check_agreement[check_agreement['signal'] == 1]['weight'].values)
    cvm = compute_cvm(correlation_probs, check_correlation['mass'].values)
    return auc_holdout, auc_public, auc_private, ks, cvm

def print_dann_metrics(model):
    auc_holdout, auc_public, auc_private, ks, cvm = cross_validation_dann(model)
    ks_threshold = 0.09
    cvm_threshold = 0.002
    print('AUC (holdout): %.3f' % auc_holdout)
    print('AUC (public leaderboard): %.3f' % auc_public)
    print('AUC (private leaderboard): %.3f' % auc_private)
    print('[%s] KS: %.2f (threshold %.2f)' % ('SUCCESS' if ks < ks_threshold else 'FAILED', ks, ks_threshold))
    print('[%s] CvM: %.4f (threshold %.3f)' % ('SUCCESS' if cvm < cvm_threshold else 'FAILED', cvm, cvm_threshold))

In [40]:
model = load_model(model_path='model_trained.json', weigths_path='model_trained.h5')
dense_1_weights, dense_2_weights, dense_3_weights, dense_4_weights, dense_5_weights, dense_6_weights = tuple([(model.get_weights()[3*i], model.get_weights()[3*i+1]) for i in range(0, 6)])

In [41]:
def categorical_crossentropy_inverse(y_true, y_pred):
    return -K.categorical_crossentropy(y_pred, y_true)

def build_dann(feature_layers=[dense_1_weights, dense_2_weights], 
               predictor_layers=[dense_3_weights, dense_4_weights, dense_5_weights, dense_6_weights], 
               train_target=True, train_domain=False, optimizer=RMSprop()):
    features_layer_1_weights, features_layer_1_biases = feature_layers[0]
    features_layer_2_weights, features_layer_2_biases = feature_layers[1]
    
    predictor_layer_1_weights, predictor_layer_1_biases = predictor_layers[0]
    predictor_layer_2_weights, predictor_layer_2_biases = predictor_layers[1]
    predictor_layer_3_weights, predictor_layer_3_biases = predictor_layers[2]
    predictor_layer_4_weights, predictor_layer_4_biases = predictor_layers[3]
    
    
    feature_generator = Sequential()
    feature_generator.add(Dense(input_dim=features_layer_1_weights.shape[0], output_dim=features_layer_1_weights.shape[1], weights=[features_layer_1_weights, features_layer_1_biases]))
    feature_generator.add(PReLU())
    feature_generator.add(Dense(input_dim=features_layer_2_weights.shape[0], output_dim=features_layer_2_weights.shape[1], weights=[features_layer_2_weights, features_layer_2_biases]))
    feature_generator.add(PReLU())
    
#     feature_generator.compile(optimizer=Nadam(), loss='categorical_crossentropy', metrics=['acc'])
#     feature_generator.trainable = False

    target_input = Input(shape=(len(features),), name='target_input')
    domain_input = Input(shape=(len(features),), name='domain_input')
    
    target_features = feature_generator(target_input)
    domain_features = feature_generator(domain_input)
    
    target_predictor = Sequential(name='target_predictor')
    target_predictor.add(Dense(input_dim=predictor_layer_1_weights.shape[0], output_dim=predictor_layer_1_weights.shape[1], weights=[predictor_layer_1_weights, predictor_layer_1_biases]))
    target_predictor.add(PReLU())
    target_predictor.add(Dense(input_dim=predictor_layer_2_weights.shape[0], output_dim=predictor_layer_2_weights.shape[1], weights=[predictor_layer_2_weights, predictor_layer_2_biases]))
    target_predictor.add(Dropout(0.09))
    target_predictor.add(PReLU())
    target_predictor.add(Dense(input_dim=predictor_layer_3_weights.shape[0], output_dim=predictor_layer_3_weights.shape[1], weights=[predictor_layer_3_weights, predictor_layer_3_biases]))
    target_predictor.add(Dropout(0.07))
    target_predictor.add(PReLU())
    target_predictor.add(Dense(input_dim=predictor_layer_4_weights.shape[0], output_dim=predictor_layer_4_weights.shape[1], activation='softmax', name='target_output', weights=[predictor_layer_4_weights, predictor_layer_4_biases]))
#     predictor.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['acc'])
    target_predictor.trainable = train_target
    
    domain_adaptor = Sequential(name='domain_adaptor')
    domain_adaptor.add(Dense(input_dim=features_layer_2_weights.shape[1], output_dim=25))
    domain_adaptor.add(Dense(10))
    domain_adaptor.add(Dense(2, name='domain_output', activation='softmax'))
#     domain_adaptor.compile(optimizer=RMSprop(), loss=categorical_crossentropy_inverse, metrics=['auc'])
    domain_adaptor.trainable = train_domain
    
    target_output = target_predictor(target_features)
    domain_output = domain_adaptor(domain_features)
    
    model = Model(inputs=[target_input, domain_input], outputs=[target_output, domain_output])

    model.compile(optimizer=optimizer, 
                  loss={'target_predictor': 'categorical_crossentropy', 'domain_adaptor': categorical_crossentropy_inverse},
                  metrics=['acc'])
    return model

In [42]:
dann_model = build_dann(train_target=False, train_domain=False)

dann_model.fit({'target_input': Xt_train, 'domain_input': Xd_2},
          {'target_predictor': np_utils.to_categorical(yt_train), 'domain_adaptor': np_utils.to_categorical(yd_2)},
          epochs=5, batch_size=64, verbose=1)

print_dann_metrics(dann_model)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=70, weights=[array([[-..., input_dim=46)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=35, weights=[array([[-..., input_dim=70)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:32: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=20, weights=[array([[ ..., input_dim=35)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:34: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=10, weights=[array([[-..., input_dim=20)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=7, weights=[array([[ ..., input_dim=10)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:40: UserWarning: Update your `Dense` call to the K

Epoch 1/5
54042/54042 [==============================] - 2s - loss: -7.3929 - target_predictor_loss: 0.6546 - domain_adaptor_loss: -8.0474 - target_predictor_acc: 0.7158 - domain_adaptor_acc: 0.5007     
Epoch 2/5
54042/54042 [==============================] - 1s - loss: -7.4590 - target_predictor_loss: 0.6039 - domain_adaptor_loss: -8.0629 - target_predictor_acc: 0.7197 - domain_adaptor_acc: 0.4998     
Epoch 3/5
54042/54042 [==============================] - 1s - loss: -7.4754 - target_predictor_loss: 0.5875 - domain_adaptor_loss: -8.0629 - target_predictor_acc: 0.7259 - domain_adaptor_acc: 0.4998     
Epoch 4/5
54042/54042 [==============================] - 2s - loss: -7.4853 - target_predictor_loss: 0.5776 - domain_adaptor_loss: -8.0629 - target_predictor_acc: 0.7377 - domain_adaptor_acc: 0.4998     
Epoch 5/5
54042/54042 [==============================] - 1s - loss: -7.4937 - target_predictor_loss: 0.5692 - domain_adaptor_loss: -8.0629 - target_predictor_acc: 0.7445 - domain_adapt

In [43]:
dann_model = build_dann(train_domain=False)

dann_model.fit({'target_input': Xt_train, 'domain_input': Xd_2},
          {'target_predictor': np_utils.to_categorical(yt_train), 'domain_adaptor': np_utils.to_categorical(yd_2)},
          epochs=5, batch_size=64, verbose=1)

print_dann_metrics(dann_model)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=70, weights=[array([[-..., input_dim=46)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=35, weights=[array([[-..., input_dim=70)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:32: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=20, weights=[array([[ ..., input_dim=35)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:34: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=10, weights=[array([[-..., input_dim=20)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=7, weights=[array([[ ..., input_dim=10)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:40: UserWarning: Update your `Dense` call to the K

Epoch 1/5
54042/54042 [==============================] - 6s - loss: -2.0797 - target_predictor_loss: 5.9833 - domain_adaptor_loss: -8.0629 - target_predictor_acc: 0.6012 - domain_adaptor_acc: 0.4998     
Epoch 2/5
54042/54042 [==============================] - 4s - loss: -1.8890 - target_predictor_loss: 6.1739 - domain_adaptor_loss: -8.0629 - target_predictor_acc: 0.6169 - domain_adaptor_acc: 0.4998     
Epoch 3/5
54042/54042 [==============================] - 4s - loss: -1.8864 - target_predictor_loss: 6.1765 - domain_adaptor_loss: -8.0629 - target_predictor_acc: 0.6168 - domain_adaptor_acc: 0.4998     
Epoch 4/5
54042/54042 [==============================] - 4s - loss: -1.8909 - target_predictor_loss: 6.1720 - domain_adaptor_loss: -8.0629 - target_predictor_acc: 0.6171 - domain_adaptor_acc: 0.4998     
Epoch 5/5
54042/54042 [==============================] - 4s - loss: -1.8899 - target_predictor_loss: 6.1730 - domain_adaptor_loss: -8.0629 - target_predictor_acc: 0.6170 - domain_adapt

In [35]:
dann_model = build_dann(train_domain=True)

dann_model.fit({'target_input': Xt_train, 'domain_input': Xd_2},
          {'target_predictor': np_utils.to_categorical(yt_train), 'domain_adaptor': np_utils.to_categorical(yd_2)},
          epochs=5, batch_size=64, verbose=1)

print_dann_metrics(dann_model)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=70, weights=[array([[ ..., input_dim=46)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=35, weights=[array([[-..., input_dim=70)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:32: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=20, weights=[array([[ ..., input_dim=35)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:34: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=10, weights=[array([[ ..., input_dim=20)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=7, weights=[array([[-..., input_dim=10)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:40: UserWarning: Update your `Dense` call to the K

Epoch 1/5
54042/54042 [==============================] - 3s - loss: -7.3532 - target_predictor_loss: 0.7097 - domain_adaptor_loss: -8.0629 - target_predictor_acc: 0.6070 - domain_adaptor_acc: 0.4998     
Epoch 2/5
54042/54042 [==============================] - 3s - loss: -7.3784 - target_predictor_loss: 0.6845 - domain_adaptor_loss: -8.0629 - target_predictor_acc: 0.6167 - domain_adaptor_acc: 0.4998     
Epoch 3/5
54042/54042 [==============================] - 2s - loss: -7.3935 - target_predictor_loss: 0.6694 - domain_adaptor_loss: -8.0629 - target_predictor_acc: 0.6170 - domain_adaptor_acc: 0.4998     
Epoch 4/5
54042/54042 [==============================] - 2s - loss: -7.4007 - target_predictor_loss: 0.6623 - domain_adaptor_loss: -8.0629 - target_predictor_acc: 0.6160 - domain_adaptor_acc: 0.4998     
Epoch 5/5
54042/54042 [==============================] - 2s - loss: -7.4027 - target_predictor_loss: 0.6602 - domain_adaptor_loss: -8.0629 - target_predictor_acc: 0.6201 - domain_adapt

Попробуем теперь использовать недотренированную модель

In [44]:
# model = load_model(model_path='model_trained.json', weigths_path='model_trained.h5')
model = load_model(model_path='model.json', weigths_path='model.h5')
dense_1_weights, dense_2_weights, dense_3_weights, dense_4_weights, dense_5_weights, dense_6_weights = tuple([(model.get_weights()[3*i], model.get_weights()[3*i+1]) for i in range(0, 6)])

In [48]:
dann_model = build_dann(feature_layers=[dense_1_weights, dense_2_weights], 
                        predictor_layers=[dense_3_weights, dense_4_weights, dense_5_weights, dense_6_weights],
                        train_domain=True)

dann_model.fit({'target_input': Xt_train, 'domain_input': Xd_2},
          {'target_predictor': np_utils.to_categorical(yt_train), 'domain_adaptor': np_utils.to_categorical(yd_2)},
          epochs=1, batch_size=64, verbose=1)

print_dann_metrics(dann_model)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:17: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=70, weights=[array([[ ..., input_dim=46)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:19: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=35, weights=[array([[-..., input_dim=70)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:32: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=20, weights=[array([[ ..., input_dim=35)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:34: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=10, weights=[array([[ ..., input_dim=20)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:37: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=7, weights=[array([[-..., input_dim=10)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:40: UserWarning: Update your `Dense` call to the K

Epoch 1/1
54042/54042 [==============================] - 6s - loss: -7.3597 - target_predictor_loss: 0.6861 - domain_adaptor_loss: -8.0458 - target_predictor_acc: 0.6150 - domain_adaptor_acc: 0.5008     
AUC (holdout): 0.735
AUC (public leaderboard): 0.728
AUC (private leaderboard): 0.732
[SUCCESS] KS: 0.07 (threshold 0.09)
[SUCCESS] CvM: 0.0009 (threshold 0.002)


Let's build network with domain and mass-adaptation layers

In [96]:
def mass_loss(y_pred, y_target):
    '''
    Analogue of CvM between prediction and masses. Minimizes sum (y_pred - 0.5) * mass
    :param y_pred - predictions
    :param y_target - mass
    '''
    return K.mean(y_pred * y_target)

def build_dann_2(feature_layers=[dense_1_weights, dense_2_weights], 
               predictor_layers=[dense_3_weights, dense_4_weights, dense_5_weights, dense_6_weights], 
               train_target=True, train_domain=False, train_mass=False, optimizer=RMSprop()):
    features_layer_1_weights, features_layer_1_biases = feature_layers[0]
    features_layer_2_weights, features_layer_2_biases = feature_layers[1]
    
    predictor_layer_1_weights, predictor_layer_1_biases = predictor_layers[0]
    predictor_layer_2_weights, predictor_layer_2_biases = predictor_layers[1]
    predictor_layer_3_weights, predictor_layer_3_biases = predictor_layers[2]
    predictor_layer_4_weights, predictor_layer_4_biases = predictor_layers[3]
    
    
    feature_generator = Sequential()
    feature_generator.add(Dense(input_dim=features_layer_1_weights.shape[0], output_dim=features_layer_1_weights.shape[1], weights=[features_layer_1_weights, features_layer_1_biases]))
    feature_generator.add(PReLU())
    feature_generator.add(Dense(input_dim=features_layer_2_weights.shape[0], output_dim=features_layer_2_weights.shape[1], weights=[features_layer_2_weights, features_layer_2_biases]))
    feature_generator.add(PReLU())
    
#     feature_generator.compile(optimizer=Nadam(), loss='categorical_crossentropy', metrics=['acc'])
#     feature_generator.trainable = False

    target_input = Input(shape=(len(features),), name='target_input')
    domain_input = Input(shape=(len(features),), name='domain_input')
    mass_input = Input(shape=(len(features),), name='mass_input')
    
    target_features = feature_generator(target_input)
    domain_features = feature_generator(domain_input)
    mass_features = feature_generator(mass_input)
    
    target_predictor = Sequential(name='target_predictor')
    target_predictor.add(Dense(input_dim=predictor_layer_1_weights.shape[0], output_dim=predictor_layer_1_weights.shape[1], weights=[predictor_layer_1_weights, predictor_layer_1_biases]))
    target_predictor.add(PReLU())
    target_predictor.add(Dense(input_dim=predictor_layer_2_weights.shape[0], output_dim=predictor_layer_2_weights.shape[1], weights=[predictor_layer_2_weights, predictor_layer_2_biases]))
    target_predictor.add(Dropout(0.09))
    target_predictor.add(PReLU())
    target_predictor.add(Dense(input_dim=predictor_layer_3_weights.shape[0], output_dim=predictor_layer_3_weights.shape[1], weights=[predictor_layer_3_weights, predictor_layer_3_biases]))
    target_predictor.add(Dropout(0.07))
    target_predictor.add(PReLU())
    target_predictor.add(Dense(input_dim=predictor_layer_4_weights.shape[0], output_dim=predictor_layer_4_weights.shape[1], activation='softmax', name='target_output', weights=[predictor_layer_4_weights, predictor_layer_4_biases]))
#     predictor.compile(optimizer=RMSprop(), loss='categorical_crossentropy', metrics=['acc'])
    target_predictor.trainable = train_target
    
    domain_adaptor = Sequential(name='domain_adaptor')
    domain_adaptor.add(Dense(input_dim=features_layer_2_weights.shape[1], output_dim=25))
    domain_adaptor.add(Dense(10))
    domain_adaptor.add(Dense(2, name='domain_output', activation='softmax'))
#     domain_adaptor.compile(optimizer=RMSprop(), loss=categorical_crossentropy_inverse, metrics=['auc'])
    domain_adaptor.trainable = train_domain
    
    mass_adaptor = Sequential(name='mass_adaptor')
    mass_adaptor.add(Dense(input_dim=features_layer_2_weights.shape[1], output_dim=10))
    mass_adaptor.add(Dense(1, name='mass_output', activation='softmax'))
    mass_adaptor.trainable = train_mass
    
    target_output = target_predictor(target_features)
    domain_output = domain_adaptor(domain_features)
    mass_output = mass_adaptor(mass_features)
    
    model = Model(inputs=[target_input, domain_input, mass_input], outputs=[target_output, domain_output, mass_output])

    model.compile(optimizer=optimizer, 
                  loss={'target_predictor': 'categorical_crossentropy', 'domain_adaptor': categorical_crossentropy_inverse, 
                        'mass_adaptor': mass_loss
                       },
                  metrics=['acc'])
    return model

In [97]:
def cross_validation_dann_2(model):
    agreement_probs = model.predict([check_agreement[features].values, np.zeros_like(check_agreement[features].values), np.zeros_like(check_agreement[features].values)])[0][:, 1]
    correlation_probs = model.predict([check_correlation[features].values, np.zeros_like(check_correlation[features].values), np.zeros_like(check_agreement[features].values)])[0][:, 1]
    auc_holdout = roc_auc_truncated(yt_test, model.predict([Xt_test, np.zeros_like(Xt_test), np.zeros_like(Xt_test)])[0][:, 1])
    auc_public = roc_auc_truncated(y_public, model.predict([X_public, np.zeros_like(X_public), np.zeros_like(X_public)])[0][:, 1])
    auc_private = roc_auc_truncated(y_private, model.predict([X_private, np.zeros_like(X_private), np.zeros_like(X_private)])[0][:, 1])
    ks = compute_ks(agreement_probs[check_agreement['signal'].values == 0],
        agreement_probs[check_agreement['signal'].values == 1],
        check_agreement[check_agreement['signal'] == 0]['weight'].values,
        check_agreement[check_agreement['signal'] == 1]['weight'].values)
    cvm = compute_cvm(correlation_probs, check_correlation['mass'].values)
    return auc_holdout, auc_public, auc_private, ks, cvm

def print_dann_2_metrics(model):
    auc_holdout, auc_public, auc_private, ks, cvm = cross_validation_dann_2(model)
    ks_threshold = 0.09
    cvm_threshold = 0.002
    print('AUC (holdout): %.3f' % auc_holdout)
    print('AUC (public leaderboard): %.3f' % auc_public)
    print('AUC (private leaderboard): %.3f' % auc_private)
    print('[%s] KS: %.2f (threshold %.2f)' % ('SUCCESS' if ks < ks_threshold else 'FAILED', ks, ks_threshold))
    print('[%s] CvM: %.4f (threshold %.3f)' % ('SUCCESS' if cvm < cvm_threshold else 'FAILED', cvm, cvm_threshold))
    
dann_2_model = build_dann_2(train_domain=True, train_mass=True)

dann_2_model.fit({'target_input': Xt_train, 'domain_input': Xd_2, 'mass_input': Xm},
          {'target_predictor': np_utils.to_categorical(yt_train), 'domain_adaptor': np_utils.to_categorical(yd_2), 'mass_adaptor': ym},
          epochs=1, batch_size=64, verbose=1)

print_dann_2_metrics(dann_2_model)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=70, weights=[array([[ ..., input_dim=46)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=35, weights=[array([[-..., input_dim=70)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:39: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=20, weights=[array([[ ..., input_dim=35)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:41: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=10, weights=[array([[ ..., input_dim=20)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:44: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=7, weights=[array([[-..., input_dim=10)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:47: UserWarning: Update your `Dense` call to the K

Epoch 1/1
54042/54042 [==============================] - 10s - loss: 1773.3797 - target_predictor_loss: 0.8499 - domain_adaptor_loss: -8.0552 - mass_adaptor_loss: 1780.5849 - target_predictor_acc: 0.6038 - domain_adaptor_acc: 0.5002 - mass_adaptor_acc: 0.0000e+00    
AUC (holdout): 0.693
AUC (public leaderboard): 0.731
AUC (private leaderboard): 0.725
[SUCCESS] KS: 0.07 (threshold 0.09)
[SUCCESS] CvM: 0.0009 (threshold 0.002)


In [98]:
dann_2_model = build_dann_2(train_domain=True, train_mass=False)

dann_2_model.fit({'target_input': Xt_train, 'domain_input': Xd_2, 'mass_input': Xm},
          {'target_predictor': np_utils.to_categorical(yt_train), 'domain_adaptor': np_utils.to_categorical(yd_2), 'mass_adaptor': ym},
          epochs=1, batch_size=64, verbose=1)

print_dann_2_metrics(dann_2_model)

/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:22: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=70, weights=[array([[ ..., input_dim=46)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:24: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=35, weights=[array([[-..., input_dim=70)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:39: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=20, weights=[array([[ ..., input_dim=35)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:41: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=10, weights=[array([[ ..., input_dim=20)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:44: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=7, weights=[array([[-..., input_dim=10)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:47: UserWarning: Update your `Dense` call to the K

Epoch 1/1
54042/54042 [==============================] - 10s - loss: 1773.2488 - target_predictor_loss: 0.7268 - domain_adaptor_loss: -8.0629 - mass_adaptor_loss: 1780.5849 - target_predictor_acc: 0.6078 - domain_adaptor_acc: 0.4998 - mass_adaptor_acc: 0.0000e+00    
AUC (holdout): 0.724
AUC (public leaderboard): 0.688
AUC (private leaderboard): 0.691
[SUCCESS] KS: 0.08 (threshold 0.09)
[SUCCESS] CvM: 0.0008 (threshold 0.002)
